In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from pythainlp.tokenize import word_tokenize
import torch
import accelerate

In [ ]:
df = pd.read_csv('../preparedata/data_train.csv')

sentences = []
labels = []
label2id = {
    "O": 0,
    "B-FOOD": 1,
    "I-FOOD": 2,
    "B-COMMAND_1": 3,
    "I-COMMAND_1": 4,
    "B-COMMAND_2": 5,
    "I-COMMAND_2": 6,
    "B-TABLE": 7,
    "I-TABLE": 8,
    "B-QUESTION": 9,
    "I-QUESTION": 10
}
id2label = {v: k for k, v in label2id.items()}

for sentence_id, group in df.groupby('SENTENCE'):
    tokens = group['WORD_TOKENIZE'].tolist()
    tags = group['TAG'].tolist()
    sentences.append(tokens)
    labels.append(tags)

encoded_labels = [[label2id[label] for label in sentence] for sentence in labels]

train_sentences, test_sentences, train_labels, test_label = train_test_split(sentences, encoded_labels, test_size=0.2)

data_traning = {
    "tokens" :train_sentences,
    "labels": train_labels
}

data_testing = {
    "tokens" :test_sentences,
    "labels": test_label
}

print("ตัวอย่าง sentence:", data_traning['labels'][0])
print("ตัวอย่าง sentence:", data_traning['tokens'][0])
print("ตัวอย่าง sentence:", data_testing['labels'][0])
print("ตัวอย่าง sentence:", data_testing['tokens'][0])

In [ ]:
model_name = "Geotrend/bert-base-th-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    # id2label=id2label,
    # label2id=label2id
)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding=True)

    label_all_tokens = True
    labels = []

    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels

    return tokenized_inputs

train_encodings = tokenize_and_align_labels(data_traning)
test_encodings = tokenize_and_align_labels(data_testing)

# สร้าง Hugging Face Dataset
train_dataset = Dataset.from_dict(train_encodings)
test_dataset = Dataset.from_dict(test_encodings)

In [ ]:
def merge_tokens_and_labels(tokens, labels):
    merged_tokens = []
    merged_labels = []

    current_word = []
    current_label = None

    for token, label in zip(tokens, labels):
        token = str(token)

        if token in ['[CLS]', '[SEP]']:
            continue

        if token.startswith("##"):
            current_word.append(token[2:])
        else:

            if current_word:
                merged_tokens.append("".join(current_word))
                merged_labels.append(current_label)

            current_word = [token]
            current_label = label

    if current_word:
        merged_tokens.append("".join(current_word))
        merged_labels.append(current_label)
    decoded_labels = [id2label[id] for id in merged_labels]

    return merged_tokens, decoded_labels

In [ ]:
import torch
import accelerate
print(accelerate.__version__)
print(torch.__version__)
print(torch.cuda.is_available())
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()
results = trainer.evaluate()
print(results)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn_crfsuite.utils import flatten

import seaborn as sns
import matplotlib.pyplot as plt

predictions = trainer.predict(test_dataset)

y_pred = np.argmax(predictions.predictions, axis=2)
y_true = test_dataset['labels']


y_pred_flat = flatten(y_pred)
y_true_flat = flatten(y_true)

sorted_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print(classification_report(y_true_flat, y_pred_flat, labels=sorted_labels, zero_division=0))

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=sorted_labels)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=sorted_labels, yticklabels=sorted_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for NER (Encoded Labels as Numbers)')
plt.show()

In [ ]:
y_true_filtered = [true_label for true_label in y_true_flat if true_label != -100]
y_pred_filtered = [pred_label for true_label, pred_label in zip(y_true_flat, y_pred_flat) if true_label != -100]

y_pred_encoded = y_pred_text = [id2label[label] for label in y_pred_filtered]
y_true_encoded = y_pred_text = [id2label[label] for label in y_true_filtered]

sorted_labels =  ['B-COMMAND_1', 'I-COMMAND_1', 'B-COMMAND_2','I-COMMAND_2', 'B-FOOD', 'I-FOOD', 'B-QUESTION', 'I-QUESTION', 'B-TABLE', 'I-TABLE']
print(classification_report(y_true_encoded, y_pred_encoded, labels=sorted_labels, zero_division=0))
cm = confusion_matrix(y_true_encoded, y_pred_encoded, labels=sorted_labels)

cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(10, 8))
sns.heatmap(cm_percentage, annot=True, fmt='.1f', xticklabels=sorted_labels, yticklabels=sorted_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix as Percentage (%)')
plt.savefig('confusion_matrix_tranformer_model.png')
plt.show()

In [ ]:
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

In [ ]:
model_path = './model'
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

text = "ขนมปังสังขยาโต๊ะ 3 แล้วก็ของโต๊ะ 4 ชั้นได้เตรียมไว้ให้เสร็จแล้วหรือยังครับผม"
word_cut = word_tokenize(text, keep_whitespace=False)

inputs = tokenizer(word_cut, truncation=True, is_split_into_words=True, padding=True, return_tensors="pt")

model.eval()

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
predicted_labels = torch.argmax(logits, dim=-1)

print(len(inputs["input_ids"][0].tolist()), len(predicted_labels[0].tolist()))

predicted_labels_list = predicted_labels[0].tolist()
inputs_list = inputs["input_ids"][0].tolist()

text, tag_ner = merge_tokens_and_labels(tokenizer.convert_ids_to_tokens(inputs_list), predicted_labels_list)
print(text)
print(tag_ner)